In [2]:
# !pip install chromadb

In [3]:
import os
database_name = "chroma_db"

In [4]:
from langchain.document_loaders import PyMuPDFLoader
# Load the PDF
loader = PyMuPDFLoader("data/RAMAYANA.pdf")
docs = loader.load()

# Each doc now contains `page_content` and `metadata`
for doc in docs:
    print(doc.page_content[:100])  # Preview text
    print(doc.metadata)            # {'source': 'data/ramayana.pdf', 'page': 0}

SRI RAMA JAYAM 
RAMAYANA FOR CHILDREN 
Compiled by  
 
Visalakshi Gopalan 
14-Apr-13 
 
 
 
For chil
{'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2013-04-14T19:39:50-07:00', 'source': 'data/RAMAYANA.pdf', 'file_path': 'data/RAMAYANA.pdf', 'total_pages': 45, 'format': 'PDF 1.5', 'title': 'RAMAYANA FOR CHILDREN', 'author': 'Sony', 'subject': 'Compiled by', 'keywords': '', 'moddate': '2013-04-14T19:39:50-07:00', 'trapped': '', 'modDate': "D:20130414193950-07'00'", 'creationDate': "D:20130414193950-07'00'", 'page': 0}
1 
 
Contents 
 
1 RAMAYANA FOR CHILDREN ...........................................................
{'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2013-04-14T19:39:50-07:00', 'source': 'data/RAMAYANA.pdf', 'file_path': 'data/RAMAYANA.pdf', 'total_pages': 45, 'format': 'PDF 1.5', 'title': 'RAMAYANA FOR CHILDREN', 'author': 'Sony', 'subject': 'Compiled by', 'keywords': '', 'moddate': '2013-04

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the documents into smaller chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "] # we can use custom separators 
)
chunks = splitter.split_documents(docs)
print(f"Total chunks created: {len(chunks)}")

Total chunks created: 215


In [7]:
# Read key from file (or load from env manually)
with open("data/key.txt", "r") as f:
    api_key = f.read().strip()

# Create embeddings for the chunks
from langchain.embeddings import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=api_key
)


/tmp/ipykernel_5046/687801729.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(


In [8]:
from langchain.vectorstores import Chroma

if os.path.exists(database_name) and os.listdir(database_name):
    # Load existing vector store
    vectorstore = Chroma(persist_directory=database_name, embedding_function=embedding_model)
    print("✅ Loaded existing vector store.")
else:
    # Create new vector store and persist
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_model,
        persist_directory=database_name
    )
    vectorstore.persist()
    print("✅ Vector store created and persisted.")


✅ Vector store created and persisted.


/tmp/ipykernel_5046/3670909715.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [9]:
query = "Who was Lord Rama?"
retrieved_docs = vectorstore.similarity_search(query, k=3)
for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Result {i+1} ---\n{doc.page_content[:500]}...")


--- Result 1 ---
2 
 
1 RAMAYANA FOR CHILDREN 
 
1.1 THE BIRTH OF RAMA 
 
Ayodhya was a magnificent city on the banks of the river Sarayu in 
Kosala Country .It had wide roads, huge buildings, beautiful parks 
and glittering shops. The people of the city lived a happy and 
contented life as they were ruled by a wonderful king called 
Dasharatha. He cared for his people very deeply. 
 
King Dasahratha had three wives, Kaushalya, Sumitra and Kaikeyi....

--- Result 2 ---
44 
 
Vashishta blessed Rama and with their consent, Rama agreed to 
become the king. 
Soon Rama was crowned the king of Ayodhya. The coronation was 
conducted on a grand scale, Rama and Sita were seated on the 
throne, Lakshmana, Bharatha and Shatrugna stood behind them 
Hanuman sat at Rama’s feet. 
People rejoiced about this happy event. The festivities continued for 
a week and Rama made generous gifts to everyone.  
Sri Rama ruled over Ayodhya for many years. People lived a happy,...

--- Result 3 ---
contented life